# LAB | Imbalanced

**Load the data**

In this challenge, we will be working with Credit Card Fraud dataset.

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv

Metadata

- **distance_from_home:** the distance from home where the transaction happened.
- **distance_from_last_transaction:** the distance from last transaction happened.
- **ratio_to_median_purchase_price:** Ratio of purchased price transaction to median purchase price.
- **repeat_retailer:** Is the transaction happened from same retailer.
- **used_chip:** Is the transaction through chip (credit card).
- **used_pin_number:** Is the transaction happened by using PIN number.
- **online_order:** Is the transaction an online order.
- **fraud:** Is the transaction fraudulent. **0=legit** -  **1=fraud**


In [3]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE

In [6]:
fraud_data = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/card_transdata.csv")
fraud_data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


**Steps:**

- **1.** What is the distribution of our target variable? Can we say we're dealing with an imbalanced dataset?
- **2.** Train a LogisticRegression.
- **3.** Evaluate your model. Take in consideration class importance, and evaluate it by selection the correct metric.
- **4.** Run **Oversample** in order to balance our target variable and repeat the steps above, now with balanced data. Does it improve the performance of our model? 
- **5.** Now, run **Undersample** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model?
- **6.** Finally, run **SMOTE** in order to balance our target variable and repeat the steps above (1-3), now with balanced data. Does it improve the performance of our model? 

In [7]:
print(fraud_data['fraud'].value_counts(normalize=True))

fraud
0.0    0.912597
1.0    0.087403
Name: proportion, dtype: float64


In [8]:
X = fraud_data.drop('fraud', axis=1)
y = fraud_data['fraud']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [11]:
y_pred = model.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

Confusion Matrix:
 [[181291   1228]
 [  6916  10565]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.96      0.99      0.98    182519
         1.0       0.90      0.60      0.72     17481

    accuracy                           0.96    200000
   macro avg       0.93      0.80      0.85    200000
weighted avg       0.96      0.96      0.96    200000

Accuracy: 0.95928


In [12]:
X_train_resampled, y_train_resampled = resample(X_train[y_train == 1], y_train[y_train == 1],
                                                replace=True,
                                                n_samples=X_train[y_train == 0].shape[0],
                                                random_state=42)

X_train_balanced = pd.concat([X_train[y_train == 0], X_train_resampled])
y_train_balanced = pd.concat([y_train[y_train == 0], y_train_resampled])

In [13]:
model.fit(X_train_balanced, y_train_balanced)
y_pred_balanced = model.predict(X_test)

In [14]:
print("Confusion Matrix (Oversampling):\n", confusion_matrix(y_test, y_pred_balanced))
print("Classification Report (Oversampling):\n", classification_report(y_test, y_pred_balanced))
print("Accuracy (Oversampling):", accuracy_score(y_test, y_pred_balanced))

Confusion Matrix (Oversampling):
 [[170395  12124]
 [   911  16570]]
Classification Report (Oversampling):
               precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

Accuracy (Oversampling): 0.934825


In [15]:
X_train_majority = X_train[y_train == 0]
y_train_majority = y_train[y_train == 0]
X_train_minority = X_train[y_train == 1]
y_train_minority = y_train[y_train == 1]

X_train_downsampled, y_train_downsampled = resample(X_train_majority, y_train_majority,
                                                    replace=False,
                                                    n_samples=X_train_minority.shape[0],
                                                    random_state=42)

X_train_balanced = pd.concat([X_train_downsampled, X_train_minority])
y_train_balanced = pd.concat([y_train_downsampled, y_train_minority])

In [16]:
model.fit(X_train_balanced, y_train_balanced)
y_pred_downsampled = model.predict(X_test)

In [17]:
print("Confusion Matrix (Undersampling):\n", confusion_matrix(y_test, y_pred_downsampled))
print("Classification Report (Undersampling):\n", classification_report(y_test, y_pred_downsampled))
print("Accuracy (Undersampling):", accuracy_score(y_test, y_pred_downsampled))

Confusion Matrix (Undersampling):
 [[170384  12135]
 [   918  16563]]
Classification Report (Undersampling):
               precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.93    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.93      0.94    200000

Accuracy (Undersampling): 0.934735


In [18]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [19]:
model.fit(X_train_smote, y_train_smote)
y_pred_smote = model.predict(X_test)

In [20]:
print("Confusion Matrix (SMOTE):\n", confusion_matrix(y_test, y_pred_smote))
print("Classification Report (SMOTE):\n", classification_report(y_test, y_pred_smote))
print("Accuracy (SMOTE):", accuracy_score(y_test, y_pred_smote))

Confusion Matrix (SMOTE):
 [[170498  12021]
 [   941  16540]]
Classification Report (SMOTE):
               precision    recall  f1-score   support

         0.0       0.99      0.93      0.96    182519
         1.0       0.58      0.95      0.72     17481

    accuracy                           0.94    200000
   macro avg       0.79      0.94      0.84    200000
weighted avg       0.96      0.94      0.94    200000

Accuracy (SMOTE): 0.93519
